In [3]:
import scipy.io as sio
import numpy as np

sc = np.array(sio.loadmat('data/raw/sCall_yeo.mat')['sCall'])
fc = np.array(sio.loadmat('data/raw/fCall_yeo.mat')['fCall'])
np.set_printoptions(suppress=True)


In [4]:
print(sc[np.where(sc == 0.0)].size)
print(sc.size)


9505
950000


In [5]:
print(sc.shape)
print(fc.shape)
print(fc[:, :, 0])

(100, 100, 95)
(100, 100, 95)
[[1.         0.46285875 0.33626531 ... 0.21197098 0.19487723 0.09873838]
 [0.46285875 1.         0.49560592 ... 0.38396022 0.20020912 0.04987621]
 [0.33626531 0.49560592 1.         ... 0.31682414 0.15361462 0.1290186 ]
 ...
 [0.21197098 0.38396022 0.31682414 ... 1.         0.52977315 0.51036752]
 [0.19487723 0.20020912 0.15361462 ... 0.52977315 1.         0.82157261]
 [0.09873838 0.04987621 0.1290186  ... 0.51036752 0.82157261 1.        ]]


In [6]:
new_fc = np.transpose(fc, (2, 0, 1))  # convert to inductive dataset
print(new_fc.shape)

(95, 100, 100)


In [7]:
torch.eye(100).float().dtype


NameError: name 'torch' is not defined

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset
from scipy.io import loadmat


class MyDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(MyDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['sCall_yeo.mat', 'fCall_yeo.mat']

    @property
    def processed_file_names(self):
        return ['processed_data.pt']

    def download(self):
        # Not required since the file is provided
        pass

    def process(self):
        # Load the SC and FC data
        sCall = loadmat(self.raw_paths[0])['sCall']  # shape (100, 100, 95)
        fCall = loadmat(self.raw_paths[1])['fCall']  # shape (100, 100, 95)
        fc_matrix = np.transpose(fCall, (2, 0, 1))  # convert to inductive dataset

        
        # Process each graph
        data_list = []
        num_nodes = sCall.shape[0]
        for i in range(sCall.shape[-1]):
            sc_matrix = torch.from_numpy(sCall[:, :, i])
            # edge_index = torch.nonzero(sc_matrix, as_tuple=False).t()
            edge_index = torch.tensor([(i, j) for i in range(num_nodes) for j in range(
                num_nodes) if i != j], dtype=torch.long).t().contiguous()
            edge_weight = sc_matrix[edge_index[0], edge_index[1]].unsqueeze(-1)
            x = torch.eye(100)  # node features

            # Convert fc matrix to a tensor and reshape it to a column vector
            y = fc_matrix[i, :, :].reshape(-1, 1) # target edge weights
            data_list.append(Data(x=x, edge_index=edge_index, edge_attr=edge_weight, y=y))

        # Save the processed data
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class GCNOuterProductModel(nn.Module):
    def __init__(self, num_nodes):
        super(GCNOuterProductModel, self).__init__()
        self.conv1 = GCNConv(100, 32)
        self.num_nodes = num_nodes
        # self.linear1 = nn.Linear(32 * num_nodes, num_nodes * num_nodes)
        self.tanh = nn.Tanh()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = torch.matmul(x, x.transpose(1, 0))
        x = F.relu(x)
        x = self.tanh(x)
        return x.view(-1, self.num_nodes * self.num_nodes)


In [ ]:
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
for data in dataloader:
    print(data)
    y = np.array(y)
    print((y.shape))
    break


DataBatch(x=[800, 100], edge_index=[2, 79200], edge_attr=[79200, 1], y=[8], batch=[800], ptr=[9])
(1, 10000)


In [ ]:
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
import torch.optim as optim

# Create the dataset
dataset = MyDataset('data/')

# Create the dataloader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Create the model and optimizer
model = GCNOuterProductModel(num_nodes=100)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
model.train()
for epoch in range(200):
    loss = 0
    for data in dataloader:
        optimizer.zero_grad()
        x, edge_index, edge_attr, y = data.x, data.edge_index, data.edge_attr, data.y
        y = torch.tensor(y).squeeze(2)
        pred = model(x, data.edge_index)
        print("Shape of output", pred.shape)
        print("Shape of target", y.shape)

        print("dtype of output", pred.dtype)
        print(type(pred))
        break
    break
        # curr_loss = F.mse_loss(pred.float(), y.float())
        # loss += curr_loss
    # loss.backward()
    # optimizer.step()
    # print('Epoch {}: Loss = {}'.format(epoch+1, loss.item()))


Shape of output torch.Size([1, 10000])
Shape of target torch.Size([1, 10000])
dtype of output torch.float32
<class 'torch.Tensor'>


In [ ]:
print(data)

DataBatch(x=[100, 100], edge_index=[2, 9900], edge_attr=[9900, 1], y=[1], batch=[100], ptr=[2])


In [ ]:
import torch.optim as optim
from torch.nn import MSELoss

model = GCNOuterProductModel(num_nodes=100)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = MSELoss()

for epoch in range(10):
    model.train()
    for i, data in dataset
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out, data.y[])
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


SyntaxError: expected ':' (2360902333.py, line 10)

In [ ]:
# vector x vector
tensor1 = torch.randn(3, 4)
tensor2 = torch.randn(4, 3)
print(tensor2.transpose(1, 0).shape)
print(torch.matmul(tensor1, tensor2).size())
# matrix x vector
tensor1 = torch.randn(3, 4)
tensor2 = torch.randn(4)
print(torch.matmul(tensor1, tensor2).size())
# batched matrix x broadcasted vector
tensor1 = torch.randn(10, 3, 4)
tensor2 = torch.randn(4)
torch.matmul(tensor1, tensor2).size()
# batched matrix x batched matrix
tensor1 = torch.randn(10, 3, 4)
tensor2 = torch.randn(10, 4, 5)
torch.matmul(tensor1, tensor2).size()
# batched matrix x broadcasted matrix
tensor1 = torch.randn(10, 3, 4)
tensor2 = torch.randn(4, 5)
torch.matmul(tensor1, tensor2).size()


torch.Size([3, 4])
torch.Size([3, 3])
torch.Size([3])


torch.Size([10, 3, 5])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

### another intresting model
class GCNOuterProductModel(nn.Module):
    def __init__(self, num_nodes):
        super(GCNOuterProductModel, self).__init__()
        self.conv1 = GCNConv(16, 32)
        self.num_nodes = num_nodes
        self.linear1 = nn.Linear(32 * num_nodes, num_nodes * num_nodes)
        self.tanh = nn.Tanh()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = x.view(-1, 32 * self.num_nodes)
        x = self.linear1(x)
        x = self.tanh(x)
        return x.view(-1, self.num_nodes, self.num_nodes)


In [14]:
import scipy.io as sio
import numpy as np
np.set_printoptions(suppress=True)

# subba_data1 = (sio.loadmat('new_data/fc_and_sc_sets.mat'))
# subba_data2 = (sio.loadmat('new_data/desikan_roi_zhengwu.mat'))
# subba_data3 = (sio.loadmat('new_data/destrieux_roi_zhengwu.mat'))

fc_data_file = (sio.loadmat('new_data/correlations_desikan_old.mat'))
sc_data_file = (sio.loadmat('new_data/scs_desikan.mat'))

In [15]:
print(sc_data_file.keys())
print(fc_data_file.keys())
print(sc_data_file['scs'].shape)
print(fc_data_file['fcs'].shape)


dict_keys(['__header__', '__version__', '__globals__', 'atlas', 'scs', 'subcortical_first', 'subject_list'])
dict_keys(['__header__', '__version__', '__globals__', 'ChosenROI_cortical', 'ChosenROI_subcortical', 'atlas', 'fcs', 'missing_data_rel2scs_patient_ids', 'subject_list'])
(87, 87, 1065)
(87, 87, 1058)


In [17]:
sc_subject = sc_data_file['subject_list']
sc_data = sc_data_file['scs']
sc_matrix_t = np.transpose(sc_data, (2, 0, 1))
type(sc_subject)

numpy.ndarray

In [18]:
def subject2matrix(subject_arr, mapping_arr):
    """
    subject_arr: array of subjects (size: n_subjects x 1)
    mapping_arr: array of matrices (size: n_subjects x n_nodes x n_nodes)
    """
    my_dict = {}
    for i in range(sc_subject.shape[0]):
        my_dict[sc_subject[i, 0]] = sc_matrix_t[i]
    return my_dict


In [19]:
def subject2matrix(subject_arr, mapping_arr):
    """
    subject_arr: array of subjects (size: n_subjects x 1)
    mapping_arr: array of matrices (size: n_subjects x n_nodes x n_nodes)
    """
    my_dict = {}
    for i in range(subject_arr.shape[0]):
        my_dict[subject_arr[i, 0]] = mapping_arr[i]
    return my_dict

sc_subject = sc_data_file['subject_list']
sc_data = sc_data_file['scs']
sc_matrix_t = np.transpose(sc_data, (2, 0, 1))
sc_dict = subject2matrix(sc_subject, sc_matrix_t)
print(len(sc_dict))

fc_subject = fc_data_file['subject_list']
fc_data = fc_data_file['fcs']
fc_matrix_t = np.transpose(fc_data, (2, 0, 1))
fc_dict = subject2matrix(fc_subject, fc_matrix_t)
print(len(fc_dict))



1065
1058


In [56]:
paired_dict = {}
for key in fc_dict.keys():
    if key in sc_dict:
        paired_dict[key] = (sc_dict[key], fc_dict[key])

### create numpy array for sc and fc matrix
### transpose them to (n_nodes, n_nodes, n_subjects)
### save them as .mat file
len(paired_dict)
# print(paired_dict)

1058

In [54]:
vals = list(paired_dict.values())
print(len(vals))
np.array(vals)[1][1].shape


1058


(87, 87)

In [46]:
import numpy as np

# example list of tuples
list_of_tuples = [(1, 2), (3, 4), (5, 6)]
print("size:", len(list_of_tuples))

# convert to numpy array
arr = np.array(list_of_tuples)
print("shape:", arr.shape)

# get separate arrays
arr1 = arr[:, 0]
arr2 = arr[:, 1]

# print arrays
print(arr1)
print(arr2)


size: 3
shape: (3, 2)
[1 3 5]
[2 4 6]


In [ ]:
for key, val in paired_dict.items():
    print(key, val[0].shape, val[1].shape)
    

100206 (87, 87) (87, 87)


In [ ]:
np.intersect1d(sub_list_sc, sub_list_fc).size
# len(comman_list)

1058

In [ ]:
# import torch
# import torch.nn as nn

# lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
# inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# # initialize the hidden state.
# hidden = (torch.randn(1, 1, 3),
#           torch.randn(1, 1, 3))
# for i in inputs:
#     # Step through the sequence one element at a time.
#     # after each step, hidden contains the hidden state.
#     out, hidden = lstm(i.view(1, 1, -1), hidden)

# # alternatively, we can do the entire sequence all at once.
# # the first value returned by LSTM is all of the hidden states throughout
# # the sequence. the second is just the most recent hidden state
# # (compare the last slice of "out" with "hidden" below, they are the same)
# # The reason for this is that:
# # "out" will give you access to all hidden states in the sequence
# # "hidden" will allow you to continue the sequence and backpropagate,
# # by passing it as an argument  to the lstm at a later time
# # Add the extra 2nd dimension
# inputs = torch.cat(inputs).view(len(inputs), 1, -1)
# hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
# out, hidden = lstm(inputs, hidden)
# print(out)
# print(hidden)
